In [4]:
import shlex, subprocess, time, itertools, os, shutil

In [5]:
def create_all_permutations(cmd_template, *args):
    for params in itertools.product(*args):
        yield cmd_template.format(*params)

def run_parallel(commands):
    if os.path.exists("out"):
        shutil.rmtree('out')
    os.makedirs("out")

    processes = []
    statuses = []
    output_files = []

    for i, cmd in enumerate(commands):
        print(f"Running command {i}: \"{cmd}\"")
        cmd_args = shlex.split(cmd)
        
        f = open(f"out/{i}.txt", "w")
        proc = subprocess.Popen(cmd_args, stdout=f)
        processes.append(proc)
        statuses.append(None)
        output_files.append(f)

    while None in statuses:
        time.sleep(5)
        for i in range(len(processes)):
            if statuses[i] == None:
                statuses[i] = processes[i].poll()
                if statuses[i] == 0:
                    print(f"Command {i} finished successfully!")
                elif statuses[i] != None:
                    print(f"Command {i} failed with status {statuses[i]}!!!!!")
    print("All processes done. Status codes:")
    print(statuses)

    for f in output_files:
        f.close()
    return statuses

def run_sequential(commands):
    statuses = []
    with open("output.txt", "w") as f:
        for cmd in commands:
            print(f"Running command: \"{cmd}\"")
            cmd_args = shlex.split(cmd)

            proc = subprocess.run(cmd_args, stdout=f)
            if proc.returncode == 0:
                print("Command finished successfully!")
            else:
                print(f"Command failed with status {proc.returncode}!!!!!")
                return
            statuses.append(proc.returncode)
    return statuses

In [7]:
templates = [
"""
python run_sac.py
    --env_name HalfCheetah-v4 --env_tasks forward backward jump --multitask_setting cds --ep_len 150
    --discount 0.99 --scalar_log_freq 1500
    -n 100000 -l 2 -s 256 -b 1500 -eb 1500
    -lr 0.0003 --init_temperature 0.1 --exp_name halfcheetah_fbj_cds{0}
    --seed 1 --cds_percentile {0}
""",
"""
python run_sac.py
    --env_name Walker2d-v4 --env_tasks forward backward jump --multitask_setting cds --ep_len 1000
    --discount 0.99 --scalar_log_freq 2000
    -n 100000 -l 2 -s 256 -b 2000 -eb 2000
    -lr 0.0003 --init_temperature 0.1 --exp_name walker_fbj_cds{0}
    --seed 1 --cds_percentile {0}
"""
]

percentiles = [.05, .1, .15, .25]
run_parallel([t.format(p) for t in templates for p in percentiles])

Running command 0: "
python run_sac.py
    --env_name HalfCheetah-v4 --env_tasks forward backward jump --multitask_setting cds --ep_len 150
    --discount 0.99 --scalar_log_freq 1500
    -n 100000 -l 2 -s 256 -b 1500 -eb 1500
    -lr 0.0003 --init_temperature 0.1 --exp_name halfcheetah_fbj_cds0.05
    --seed 1 --cds_percentile 0.05
"
Running command 1: "
python run_sac.py
    --env_name HalfCheetah-v4 --env_tasks forward backward jump --multitask_setting cds --ep_len 150
    --discount 0.99 --scalar_log_freq 1500
    -n 100000 -l 2 -s 256 -b 1500 -eb 1500
    -lr 0.0003 --init_temperature 0.1 --exp_name halfcheetah_fbj_cds0.1
    --seed 1 --cds_percentile 0.1
"
Running command 2: "
python run_sac.py
    --env_name HalfCheetah-v4 --env_tasks forward backward jump --multitask_setting cds --ep_len 150
    --discount 0.99 --scalar_log_freq 1500
    -n 100000 -l 2 -s 256 -b 1500 -eb 1500
    -lr 0.0003 --init_temperature 0.1 --exp_name halfcheetah_fbj_cds0.15
    --seed 1 --cds_percentile 0

/home/kev/miniconda3/envs/cs285-proj/lib/python3.7/site-packages/gym/core.py:318: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  "Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future."
/home/kev/miniconda3/envs/cs285-proj/lib/python3.7/site-packages/gym/wrappers/step_api_compatibility.py:40: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  "Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in

Command 3 finished successfully!


 95%|█████████▍| 94904/100000 [3:54:57<10:06,  8.40it/s]

Command 2 finished successfully!


...

 99%|█████████▊| 98573/100000 [4:03:51<01:37, 14.61it/s]

Command 5 finished successfully!


100%|██████████| 100000/100000 [4:05:28<00:00,  6.79it/s]


Command 4 finished successfully!
All processes done. Status codes:
[0, 0, 0, 0, 0, 0, 0, 0]


[0, 0, 0, 0, 0, 0, 0, 0]